In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from preprocessing import preprocess

In [17]:
data = pd.read_csv("train.csv", index_col=0)

In [18]:
data.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312


In [19]:
data.isna().sum()

Brand                   9705
Material                8347
Size                    6595
Compartments               0
Laptop Compartment      7444
Waterproof              7050
Style                   7970
Color                   9950
Weight Capacity (kg)     138
Price                      0
dtype: int64

In [20]:
data = preprocess(data)
data.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price,Weight Capacity (kg)_Missing,Brand_Missing,Material_Missing,Size_Missing,Laptop Compartment_Missing,Waterproof_Missing,Style_Missing,Color_Missing
id,,,,,,,,,,,,,,,,,,
0,Jansport,Leather,Medium,7.0,1,0,Tote,Black,11.611723,112.15875,0,0,0,0,0,0,0,0
1,Jansport,Canvas,Small,10.0,1,1,Messenger,Green,27.078537,68.88056,0,0,0,0,0,0,0,0
2,Under Armour,Leather,Small,2.0,1,0,Messenger,Red,16.643760,39.17320,0,0,0,0,0,0,0,0
3,Nike,Nylon,Small,8.0,1,0,Messenger,Green,12.937220,80.60793,0,0,0,0,0,0,0,0
4,Adidas,Canvas,Medium,1.0,1,1,Messenger,Green,17.749338,86.02312,0,0,0,0,0,0,0,0


In [21]:
numerical_columns = data.select_dtypes(include='number').columns
categorical_columns = data.select_dtypes(exclude='number').columns

In [22]:
print(f"{(len(numerical_columns) + len(categorical_columns))} = {len(data.columns)}")

18 = 18


In [23]:
data.columns

Index(['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price',
       'Weight Capacity (kg)_Missing', 'Brand_Missing', 'Material_Missing',
       'Size_Missing', 'Laptop Compartment_Missing', 'Waterproof_Missing',
       'Style_Missing', 'Color_Missing'],
      dtype='object')

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    OrdinalEncoder,
)
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_validate

In [25]:
ohe_columns = [
    "Brand",
    "Material",
    "Size",
    #"Compartments",
    "Style",
    "Color"
]

columns_to_scale = [
    "Compartments",
    "Weight Capacity (kg)"
]

In [26]:
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
std = StandardScaler()

In [27]:
column_transformer = ColumnTransformer(
    transformers=[
        ("OneHotEncoding", ohe, ohe_columns),
        ("Standardization", std, columns_to_scale)
    ],
    remainder="passthrough",
    n_jobs=-1,
    verbose=3,
    verbose_feature_names_out=False
).set_output(transform="pandas")

In [28]:
pipeline = Pipeline(
    [
        ("ColumnTransformer", column_transformer),
    ]
)

In [29]:
pipeline.fit_transform(data)

,Brand_Adidas,Brand_Inconnu,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Inconnu,Material_Leather,Material_Nylon,...,Waterproof,Price,Weight Capacity (kg)_Missing,Brand_Missing,Material_Missing,Size_Missing,Laptop Compartment_Missing,Waterproof_Missing,Style_Missing,Color_Missing
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,112.15875,0,0,0,0,0,0,0,0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1,68.88056,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0,39.17320,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,80.60793,0,0,0,0,0,0,0,0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1,86.02312,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,129.99749,0,0,0,0,0,0,0,0
299996,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1,19.85819,0,0,0,0,0,0,0,0
299997,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1,111.41364,0,0,0,0,0,0,0,0
